In [1]:
# Import Libraries 
import pandas as pd 
import numpy as np 
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly
import plotly.express as ex
import plotly.graph_objs as go

from sklearn.model_selection import train_test_split,cross_val_score,RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler

from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score

from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
train= pd.read_csv('Resources_2/train.csv',index_col='PassengerId')
test= pd.read_csv('Resources_2/test.csv',index_col='PassengerId')

In [3]:
train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [4]:
test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [5]:
# generate test/ training set
# Drop 'Survived' column 
x_train,x_test,y_train,y_test=train_test_split(train.drop('Survived',axis=1),train['Survived'],test_size=.2) 

In [6]:
# X_Train
# replace NA age with median age grouped by passenger class   
x_train['Age']=x_train['Age'].fillna(x_train.groupby(['Pclass'])['Age'].transform('median'))
#
x_train['Embarked']=x_train['Embarked'].fillna('S')
# X_Test                                       
x_test['Age']=x_test['Age'].fillna(x_test.groupby(['Pclass'])['Age'].transform('median'))
x_test['Embarked']=x_test['Embarked'].fillna('S')
# 
# X_Train
x_train['fare_group']=pd.qcut(x_train['Fare'],5)
# X_Test
x_test['fare_group']=pd.qcut(x_test['Fare'],5)


# convert sex to numerical value
# X_Train
x_train['Sex']=x_train['Sex'].replace({'male':0,'female':1})
# X_Test
x_test['Sex']=x_test['Sex'].replace({'male':0,'female':1})

# # Drop 'Embarked' and generate a column of numerical values called 'fare_group'
# X_Train
x_train=pd.get_dummies(x_train,columns=['Embarked','fare_group'],drop_first=True)
# X_Test
x_test=pd.get_dummies(x_test,columns=['Embarked','fare_group'],drop_first=True)

## Generate a column holding passengers as numerical values based on'title'
# convert passengers into numerical values using pro-noun and title 
## X_Train
title=[]
for i in range(len(x_train)):
    title.append(x_train['Name'].iloc[i].split(',')[1].split('.')[0])
x_train['title']=title
# convert pro-nouns to 'Who Cares'
x_train['title']=x_train['title'].replace([' Col',' Major',' Mlle',' Ms',' the Countess',' Capt',' Lady',' Mme',' Sir',' Don',' Jonkheer'],'Who Cares')
# convert 'Who Cares' to numerical values
x_train['title']=x_train['title'].replace({' Mr':1,' Mrs':2,' Miss':3,' Master':4,'Who Cares':5,' Dr':6,' Rev':5})
## X_Test
title=[]
for i in range(len(x_test)):
    title.append(x_test['Name'].iloc[i].split(',')[1].split('.')[0])
x_test['title']=title
#
x_test['title']=x_test['title'].replace([' Col',' Major',' Mlle',' Ms',' the Countess',' Capt',' Lady',' Mme',' Sir',' Don',' Jonkheer'],'Who Cares')
#
x_test['title']=x_test['title'].replace({' Mr':1,' Mrs':2,' Miss':3,' Master':4,'Who Cares':5,' Dr':6,' Rev':5})

# combine 'SibSp' and 'Parch' into a column
# X_Train
x_train['total']=x_train['SibSp']+x_train['Parch']
# X_Test
x_test['total']=x_test['SibSp']+x_test['Parch']

# Drop non-numerical columns
# X_Train
x_train.drop(['Name','Ticket','Cabin','SibSp','Parch'],axis=1,inplace=True)
# X_Test
x_test.drop(['Name','Ticket','Cabin','SibSp','Parch'],axis=1,inplace=True)

# get the square root of 'Fare' 
# X_Train
x_train['Fare']=np.sqrt(x_train['Fare'])
# X_Test
x_test['Fare']=np.sqrt(x_test['Fare'])

In [7]:
x_train

,Pclass,Sex,Age,Fare,Embarked_Q,Embarked_S,"fare_group_(7.854, 10.5]","fare_group_(10.5, 20.83]","fare_group_(20.83, 39.0]","fare_group_(39.0, 512.329]",title,total
PassengerId,,,,,,,,,,,,
228,3,0,20.5,2.692582,0,1,0,0,0,0,1,0
492,3,0,21.0,2.692582,0,1,0,0,0,0,1,0
596,3,0,36.0,4.914265,0,1,0,0,1,0,1,2
206,3,1,2.0,3.234579,0,1,1,0,0,0,3,1
232,3,0,29.0,2.788369,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
699,1,0,49.0,10.530114,0,0,0,0,0,1,1,2
521,1,1,30.0,9.669540,0,1,0,0,0,1,3,0
704,3,0,25.0,2.782391,1,0,0,0,0,0,1,0


In [8]:
x_test

,Pclass,Sex,Age,Fare,Embarked_Q,Embarked_S,"fare_group_(7.879, 10.5]","fare_group_(10.5, 26.0]","fare_group_(26.0, 48.062]","fare_group_(48.062, 512.329]",title,total
PassengerId,,,,,,,,,,,,
226,3,0,22.0,3.057777,0,1,1,0,0,0,1,0
14,3,0,39.0,5.592406,0,1,0,0,1,0,1,6
250,2,0,54.0,5.099020,0,1,0,1,0,0,5,1
694,3,0,25.0,2.687936,0,0,0,0,0,0,1,0
166,3,0,9.0,4.530453,0,1,0,1,0,0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...
567,3,0,19.0,2.809947,0,1,1,0,0,0,1,0
791,3,0,24.0,2.783882,1,0,0,0,0,0,1,0
281,3,0,65.0,2.783882,1,0,0,0,0,0,1,0


In [9]:
# Test 
# replace NA age with mean age grouped by passenger class  
test['Age']=test['Age'].fillna(test.groupby(['Pclass'])['Age'].transform('mean'))

# replace NA 'Fare' with median 'Fare'
test['Fare']=test['Fare'].fillna(test['Fare'].median())

# convert 'sex' into numerical values
test['Sex']=test['Sex'].replace({'male':0,'female':1})
#                                 
test['fare_group']=pd.qcut(test['Fare'],5)

# Drop 'Embarked' and generate a column of numerical values called 'fare_group'
test=pd.get_dummies(test, columns=['Embarked','fare_group'], drop_first=True)

## Generate a column holding passengers as numerical values based on'title'
# convert passengers into numerical values using pro-noun and title 
title=[]
for i in range(len(test)):
    title.append(test['Name'].iloc[i].split(',')[1].split('.')[0])
test['title']=title
test['title']=test['title'].replace([' Col',' Major',' Mlle',' Ms',' the Countess',' Capt',' Lady',' Mme',' Sir',' Don',' Jonkheer',' Dona'],'Who Cares')
test['title']=test['title'].replace({' Mr':1,' Mrs':2,' Miss':3,' Master':4,'Who Cares':5,' Dr':6,' Rev':5})
# combine 'SibSp' and 'Parch' into a column
test['total']=test['SibSp']+test['Parch']
# get the square root of 'Fare'                                
test['Fare']=np.sqrt(test['Fare'])
# Drop non-numerical columns                                 
test.drop(['Name','Ticket','Cabin','SibSp','Parch'],axis=1,inplace=True)

In [10]:
test

,Pclass,Sex,Age,Fare,Embarked_Q,Embarked_S,"fare_group_(7.796, 11.342]","fare_group_(11.342, 21.196]","fare_group_(21.196, 46.34]","fare_group_(46.34, 512.329]",title,total
PassengerId,,,,,,,,,,,,
892,3,0,34.500000,2.798071,1,0,1,0,0,0,1,0
893,3,1,47.000000,2.645751,0,1,0,0,0,0,2,1
894,2,0,62.000000,3.112475,1,0,1,0,0,0,1,0
895,3,0,27.000000,2.943213,0,1,1,0,0,0,1,0
896,3,1,22.000000,3.505353,0,1,0,1,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,0,24.027945,2.837252,0,1,1,0,0,0,1,0
1306,1,1,39.000000,10.435516,0,0,0,0,0,1,5,0
1307,3,0,38.500000,2.692582,0,1,0,0,0,0,1,0


In [11]:
# Train
# replace NA age with mean age grouped by passenger class  
train['Age']=train['Age'].fillna(train.groupby(['Pclass'])['Age'].transform('mean'))

# replace NA 'Fare' with median 'Fare'
train['Fare']=train['Fare'].fillna(train['Fare'].median())

# convert 'sex' into numerical values
train['Sex']=train['Sex'].replace({'male':0,'female':1})

#                                 
train['fare_group']=pd.qcut(train['Fare'],5)

# Drop 'Embarked' and generate a column of numerical values called 'fare_group'
train=pd.get_dummies(train,columns=['Embarked','fare_group'],drop_first=True)

## Generate a column holding passengers as numerical values based on'title'
# convert passengers into numerical values using pro-noun and title 
title=[]
for i in range(len(train)):
    title.append(train['Name'].iloc[i].split(',')[1].split('.')[0])
train['title']=title
train['title']=train['title'].replace([' Col',' Major',' Mlle',' Ms',' the Countess',' Capt',' Lady',' Mme',' Sir',' Don',' Jonkheer',' Dona'],'Who Cares')
train['title']=train['title'].replace({' Mr':1,' Mrs':2,' Miss':3,' Master':4,'Who Cares':5,' Dr':6,' Rev':5})

# combine 'SibSp' and 'Parch' into a column
train['total']=train['SibSp']+train['Parch']
# get the square root of 'Fare'                                
train['Fare']=np.sqrt(train['Fare'])
# Drop non-numerical columns                                 
train.drop(['Name','Ticket','Cabin','SibSp','Parch'],axis=1,inplace=True)

In [12]:
train

,Survived,Pclass,Sex,Age,Fare,Embarked_Q,Embarked_S,"fare_group_(7.854, 10.5]","fare_group_(10.5, 21.679]","fare_group_(21.679, 39.688]","fare_group_(39.688, 512.329]",title,total
PassengerId,,,,,,,,,,,,,
1,0,3,0,22.00000,2.692582,0,1,0,0,0,0,1,1
2,1,1,1,38.00000,8.442944,0,0,0,0,0,1,2,1
3,1,3,1,26.00000,2.815138,0,1,1,0,0,0,3,0
4,1,1,1,35.00000,7.286975,0,1,0,0,0,1,2,1
5,0,3,0,35.00000,2.837252,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,0,27.00000,3.605551,0,1,0,1,0,0,5,0
888,1,1,1,19.00000,5.477226,0,1,0,0,1,0,3,0
889,0,3,1,25.14062,4.842520,0,1,0,0,1,0,3,3


In [13]:
num_train = "passenger_train.csv"
train.to_csv(num_train, index_label=None)

num_test = "passenger_test.csv"
test.to_csv(num_test, index_label=None)

num_x_train = "passenger_x_train.csv"
x_train.to_csv(num_x_train, index_label=None)

num_x_test = "passenger_x_test.csv"
x_test.to_csv(num_x_test, index_label=None)